In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold

from sklearn.multiclass import OneVsRestClassifier

from scipy.spatial.distance import cdist

import re

from sklearn.neural_network import MLPClassifier

from keras.models import Sequential
from keras.layers import Dense, Dropout

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

Using TensorFlow backend.


In [2]:
def get_output(better = [False, "noun", 5]):
    if better[0]:
        y = pd.read_csv("better/better_train_{}_tags_{}.csv".format(better[2], better[1]))
    else:
        y = pd.read_csv("processed_tags.csv")
        y.drop(["Name"], axis=1, inplace = True)
    return np.array(y)

In [3]:
def get_data(bad_tags = [False, "noun"], fc = False, pool = False):
    if bad_tags[0]:
        train_lines = np.array([' '.join(f.split()) for i in range(10000) 
                      for f in open("features_train_prepossesing_{}/".format(bad_tags[1])
                                    + str(i) 
                                    + ".txt")])
        unique_words = list(set([item for sublist in train_lines.ravel() for item in sublist.split()]))

        count_vect = TfidfVectorizer(vocabulary = unique_words)
        X_train = count_vect.fit_transform(train_lines)
        X_train = X_train.toarray()

        test_lines = np.array([' '.join(f.split()) for i in range(2000) 
                      for f in open("features_test_prepossesing_{}/".format(bad_tags[1])
                                    + str(i) 
                                    + ".txt")])

        X_test = count_vect.transform(test_lines)
        X_test = X_test.toarray()

    elif fc:
        X_train = pd.read_csv("data/features_train/features_resnet1000_train.csv", 
                              header = None, names = ["Neuron_" + str(i) for i in range(1000)])
        indices = X_train.index
        new_indices = [int(re.findall(r'\d+', ind)[0]) for ind in indices]
        X_train = np.array(pd.DataFrame(np.array(X_train), columns=X_train.columns, index=new_indices).sort_index())
        
        X_test = pd.read_csv("data/features_test/features_resnet1000_test.csv", 
                             header = None, names = ["Neuron_" + str(i) for i in range(1000)])
        indices = X_test.index
        new_indices = [int(re.findall(r'\d+', ind)[0]) for ind in indices]
        X_test = np.array(pd.DataFrame(np.array(X_test), columns=X_test.columns, index=new_indices).sort_index())
    
    elif pool:
        X_train = pd.read_csv("data/features_train/features_resnet1000intermediate_train.csv", 
                              header = None, names = ["Neuron_" + str(i) for i in range(2048)])
        indices = X_train.index
        new_indices = [int(re.findall(r'\d+', ind)[0]) for ind in indices]
        X_train = np.array(pd.DataFrame(np.array(X_train), columns=X_train.columns, index=new_indices).sort_index())
        
        X_test = pd.read_csv("data/features_test/features_resnet1000intermediate_test.csv", 
                             header = None, names = ["Neuron_" + str(i) for i in range(2048)])
        indices = X_test.index
        new_indices = [int(re.findall(r'\d+', ind)[0]) for ind in indices]
        X_test = np.array(pd.DataFrame(np.array(X_test), columns=X_test.columns, index=new_indices).sort_index())
    
    else:
        print("Set Parameters")
        
    return X_train, X_test

In [7]:
def make_better_tags(k, words_type = "noun", product=True):
    X_train, X_test = get_data(bad_tags = [True, words_type])
    
    train_lines = np.array([' '.join(f.split()) for i in range(10000) 
                      for f in open("features_train_prepossesing_{}/".format(words_type)
                                    + str(i) 
                                    + ".txt")])
    
    unique_words = list(set([item for sublist in train_lines.ravel() for item in sublist.split()]))
    
    unique_tags = []
    for arr in X_train:
        tags = [i for i in arr.argsort()[-k:][::-1]]

        for i in tags:
            if unique_words[i] not in unique_tags:
                unique_tags.append(unique_words[i])
    
    print(len(unique_tags))
    
    d = dict([(tag, 0) for tag in unique_tags])
    
    better_tags = []
    for arr in X_train:
        d_sample = d.copy()

        for i in arr.argsort():
            if (arr[i]>0) and (unique_words[i] in unique_tags):
                d_sample[unique_words[i]] = arr[i]
        better_tags.append(list(d_sample.values()))

    print(len(better_tags[0]))

    pd.DataFrame(better_tags, 
                 columns=unique_tags).to_csv("better/better_train_{}_tags_{}.csv".format(k, words_type), 
                                                          index=False)
    
    better_tags = []
    for arr in X_test:
        d_sample = d.copy()

        for i in arr.argsort():
            if (arr[i]>0) and (unique_words[i] in unique_tags):
                d_sample[unique_words[i]] = arr[i]
        better_tags.append(list(d_sample.values()))

    print(len(better_tags[0]))

    pd.DataFrame(better_tags, columns=unique_tags).to_csv("better/better_test_{}_tags_{}.csv".format(k, words_type), 
                                                          index=False)

## Test models

In [5]:
def test_nn(X_train, y, add_tags=True):
    if add_tags:
        tags_train = pd.read_csv("processed_tags.csv")
        tags_train.drop(["Name"], axis=1, inplace = True)
        tags_train = np.array(tags_train)

        X_train = np.hstack((X_train, tags_train))

    print("# of input features {}".format(X_train.shape[1]))
        
    folds = 3
    skf = KFold(n_splits=folds)
    accuracy = []
    
    for train_index, test_index in skf.split(X_train, y):
        print("SPLIT")
        x_train, x_test = X_train[train_index], X_train[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = Sequential()
        model.add(Dense(500, input_dim=x_train.shape[1], init='normal', activation='relu'))
#         model.add(Dropout(0.2))
        model.add(Dense(y_train.shape[1], init='normal', activation='softmax'))

        # Compile model     #logarithmic  loss     #method
        model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', 
                      metrics=['accuracy'])

        # Fit the model
        model.fit(x_train, y_train, 
                  validation_data=(x_test, y_test),
                  nb_epoch=15, batch_size=100,
                  verbose=1)
        
        accuracy.append(model.evaluate(x_test, y_test)[1])
        print()

    print("Results")
    print("The average accuracy Neural Network is {}".format(np.mean(accuracy)))

In [6]:
make_better_tags(10, words_type="adj&noun&verb", product=True)

TypeError: make_better_tags() got an unexpected keyword argument 'product'

In [9]:
X_train, X_test = get_data(bad_tags = [True, "adj&noun&verb"])

train_lines = np.array([' '.join(f.split()) for i in range(10000) 
                  for f in open("features_train_prepossesing_{}/".format("adj&noun&verb")
                                + str(i) 
                                + ".txt")])

unique_words = list(set([item for sublist in train_lines.ravel() for item in sublist.split()]))

In [11]:
unique_tags = []
for arr in X_train:
    tags = [i for i in arr.argsort()[-10:][::-1]]

    for i in tags:
        if unique_words[i] not in unique_tags:
            unique_tags.append(unique_words[i])

print(len(unique_tags))

7771


In [12]:
unique_tags = []
for arr in X_train:
    tags = [i for i in arr.argsort()[-10:][::-1]]

    for i in tags:
        if unique_words[i] not in unique_tags:
            unique_tags.append(unique_words[i])

print(len(unique_tags))

d = dict([(tag, 0) for tag in unique_tags])

better_tags = []
for arr in X_train:
    d_sample = d.copy()

    for i in arr.argsort():
        if (arr[i]>0) and (unique_words[i] in unique_tags):
            d_sample[unique_words[i]] = arr[i]
    better_tags.append(list(d_sample.values()))

7771


In [17]:
len(better_tags[0])**2

60388441

In [16]:
np.outbetter_tags[0]

[0.65366438058304799,
 0.38684613176860422,
 0.26856400428191401,
 0.24580630343919188,
 0.23046369082399695,
 0.1857727303226801,
 0.17744107005270038,
 0.17051461233478074,
 0.15802486846267014,
 0.14282087538639421,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.062562594500189306,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.119160686290293,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

In [ ]:
np.out

In [7]:
# test_rf(X_train, y)

In [8]:
# X_train, X_test = get_data(pool=True)
# y = get_output(better=[True, "adj&noun", 5])

# test_nn(X_train, y, add_tags=True)

## Predictions

In [9]:
def nn_predict_description(add_tags=False, better=[True, "adj&noun", 5]):
    X_train, X_test = get_data(pool=True)
    y = get_output(better=better)
    
    if add_tags:
        tags_train = pd.read_csv("processed_tags.csv")
        tags_train.drop(["Name"], axis=1, inplace = True)
        tags_train = np.array(tags_train)
               
        X_train = np.hstack((X_train, tags_train))
        
        
        tags_test = pd.read_csv("processed_tags_test.csv")
        tags_test.drop(["Name"], axis=1, inplace = True)
        tags_test = np.array(tags_test)
                
        X_test = np.hstack((X_test, tags_test))
    
    print("# of input features {}".format(X_train.shape[1]))

    model = Sequential()
    model.add(Dense(500, input_dim=X_train.shape[1], init='normal', activation='relu'))
    model.add(Dense(y.shape[1], init='normal', activation='softmax'))
    
    # Compile model     #logarithmic  loss     #method
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    # Fit the model
    model.fit(X_train, y, 
              nb_epoch=13, batch_size=100,
              verbose=1)

    predicted_tags = model.predict(X_test)
    
    columns = pd.read_csv("better/better_train_{}_tags_{}.csv".format(better[2], better[1])).columns
    
    pd.DataFrame(predicted_tags, 
                 columns=columns).to_csv("Predicted/CNN_{}_{}_predicted_tags.csv".format(better[2], better[1]), 
                                         index=False)

In [10]:
def MLP_predict_description(add_tags=False, better=[True, "adj&noun", 5]):
    X_train, X_test = get_data(pool=True)
    y = get_output(better=better)
    
    if add_tags:
        tags_train = pd.read_csv("processed_tags.csv")
        tags_train.drop(["Name"], axis=1, inplace = True)
        tags_train = np.array(tags_train)
               
        X_train = np.hstack((X_train, tags_train))
        
        tags_test = pd.read_csv("processed_tags_test.csv")
        tags_test.drop(["Name"], axis=1, inplace = True)
        tags_test = np.array(tags_test)
                
        X_test = np.hstack((X_test, tags_test))
    
    y[y != 0] = 1

    model = MLPClassifier(hidden_layer_sizes=(500), verbose=True, max_iter=30, warm_start=True)
    model.fit(X_train, y)
    
    predicted_tags = model.predict(X_test)
    
    columns = pd.read_csv("better/better_train_{}_tags_{}.csv".format(better[2], better[1])).columns
    
    pd.DataFrame(predicted_tags, 
                 columns=columns).to_csv("Predicted/MLP_{}_{}_predicted_tags.csv".format(better[2], better[1]), 
                                         index=False)

In [11]:
def make_output(mlp=False, nn=False, pls=False, better=[True, "adj&noun", 5]):
    
    if nn == True:
        groud_truth = pd.read_csv("better/better_test_{}_tags_{}.csv".format(better[2], better[1]))
        predicted = pd.read_csv("Predicted/CNN_{}_{}_predicted_tags.csv".format(better[2], better[1]))

        simi = cdist(groud_truth, predicted, metric='cosine')
        pd.DataFrame(simi, 
                     columns=list(range(2000))).to_csv("similarity/NN_{}_pool+tags_similarity.csv".format(better[1]), 
                                                             index=False)
        
        rows = []
        k=0
        for row in simi:
            top20 = row.argsort()[:20]
            rows.append([str(k) + ".txt", " ".join([str(value) + ".jpg" for value in top20])])
            k+=1
        pd.DataFrame(rows, 
                     columns=['Descritpion_ID', 'Top_20_Image_IDs']).to_csv("Submissions/CNN_{}_submission.csv".format(better[1]), 
                                                                            index=False)
    elif mlp == True:
        groud_truth = pd.read_csv("better/better_test_{}_tags_{}.csv".format(better[2], better[1]))
        predicted = pd.read_csv("Predicted/MLP_{}_{}_predicted_tags.csv".format(better[2], better[1]))

        simi = cdist(groud_truth, predicted, metric='cosine')
        pd.DataFrame(simi, 
                     columns=list(range(2000))).to_csv("similarity/MLP_{}_pool+tags_similarity.csv".format(better[1]), 
                                                             index=False)
        
        rows = []
        k=0
        for row in simi:
            top20 = row.argsort()[:20]
            rows.append([str(k) + ".txt", " ".join([str(value) + ".jpg" for value in top20])])
            k+=1
        pd.DataFrame(rows, 
                     columns=['Descritpion_ID', 'Top_20_Image_IDs']).to_csv("Submissions/MLP_{}_submission.csv".format(better[1]), 
                                                                            index=False)
    elif pls == True:
        groud_truth = pd.read_csv("better/better_test_{}_tags_{}.csv".format(better[2], better[1]))
        predicted = pd.read_csv("Predicted/PLS_{}_{}_predicted_tags.csv".format(better[2], better[1]))

        simi = cdist(groud_truth, predicted, metric='cosine')
        pd.DataFrame(simi, 
                     columns=list(range(2000))).to_csv("similarity/PLS_{}_fc_similarity.csv".format(better[1]), 
                                                             index=False)
        
        rows = []
        k=0
        for row in simi:
            top20 = row.argsort()[:20]
            rows.append([str(k) + ".txt", " ".join([str(value) + ".jpg" for value in top20])])
            k+=1
        pd.DataFrame(rows, 
                     columns=['Descritpion_ID', 'Top_20_Image_IDs']).to_csv("Submissions/PLS_{}_submission.csv".format(better[1]), 
                                                                            index=False)
    else:
        1+1
    
    return rows

In [12]:
def make_enseble(models):
    matrices = []
    for key, weight in models.items():
        if key == "best":
            d = pd.read_csv("similarity/NN_best_pool+tags_similarity.csv").drop(["Unnamed: 0"], axis=1)
        elif key == "PLS":
            d = pd.read_csv("similarity/{}_{}_fc_similarity.csv".format(key, weight[0]))
        elif key == "TOP":
            d = pd.read_csv("similarity/NN_merge_simi.csv").drop(["Unnamed: 0"], axis=1)
        else:
            d = pd.read_csv("similarity/{}_{}_pool+tags_similarity.csv".format(key, weight[0]))
        matrices.append(weight[1]*np.array(d))

    matrices = np.array(matrices)
    
    simi = sum(matrices)
    
    images = []
    k=0
    for row in simi:
        top20 = row.argsort()[:20]
        images.append([str(k) + ".txt", " ".join([str(value) + ".jpg" for value in top20])])
        k+=1
        
    pd.DataFrame(images, 
                 columns=['Descritpion_ID', 'Top_20_Image_IDs']).to_csv("Submissions/EN_{}_submission.csv".format(list(models.keys())), 
                                                                        index=False)
    
    return images

## MLP

In [13]:
# MLP_predict_description(add_tags=True, better=[True, "adj&noun", 5])

## Neural Net

In [14]:
# nn_predict_description(add_tags=True, better=[True, "adj&noun", 5])

In [15]:
# images = make_output(mlp=True, better=[True, "adj&noun", 5])

In [16]:
# images = make_enseble({ "PLS": ["adj&noun&verb", 0.15],
#                       "TOP": ["", 0.85]})

In [17]:
# [ 0.86773841  0.06454957]

In [18]:
# image_to_show = 8

# output_statement = images[image_to_show][1]

# row, col = 5,5

# f, ax = plt.subplots(nrows=row, ncols=col, figsize=(30,30))
# for index, image_1 in enumerate(output_statement.split()):
#     if index == row*col:
#         break
#     img=mpimg.imread('data/images_test/' + image_1)
#     ax[int(index/row), index%col].set_title(image_1, fontsize=25)
#     ax[int(index/row), index%col].imshow(img)

## PLS

In [19]:
from sklearn.cross_decomposition import PLSRegression

def PLS_predict_description(better):
    X_train, X_test = get_data(pool=True)
    y = get_output(better=better)
    
    y[y != 0] = 1

    model = PLSRegression(n_components=200)
    model.fit(X_train, y)
    
    predicted_tags = model.predict(X_test)
    
    columns = pd.read_csv("better/better_train_{}_tags_{}.csv".format(better[2], better[1])).columns
    
    pd.DataFrame(predicted_tags, 
                 columns=columns).to_csv("Predicted/PLS_{}_{}_predicted_tags.csv".format(better[2], better[1]), 
                                         index=False)
    
# PLS_predict_description(better=[True, "adj&noun&verb", 5])

In [20]:
# images = make_output(pls=True, better=[True, "adj&noun&verb", 5])

In [21]:
# image_to_show = 2

# output_statement = images[image_to_show][1]

# row, col = 5,5

# f, ax = plt.subplots(nrows=row, ncols=col, figsize=(30,30))
# for index, image_1 in enumerate(output_statement.split()):
#     if index == row*col:
#         break
#     img=mpimg.imread('data/images_test/' + image_1)
#     ax[int(index/row), index%col].set_title(image_1, fontsize=25)
#     ax[int(index/row), index%col].imshow(img)

## Tunning

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

def get_simi_for_deb():
    X_train, X_test = get_data(pool=True)
    y = get_output(better=[True, "adj&noun", 5])

    tags_train = pd.read_csv("processed_tags.csv")
    tags_train.drop(["Name"], axis=1, inplace = True)
    tags_train = np.array(tags_train)

    X_train = np.hstack((X_train, tags_train))
    
    X_train = pd.DataFrame(X_train, columns=list(range(X_train.shape[1])))
    
    X_Train, X_Val, Y_train, Y_Val = train_test_split(X_train, y, train_size=0.8)
    
    gt_files = list(X_Val.index)
    
    file = open("Tunning/gt.txt","w") 
 
    for g in gt_files:
        file.write(str(g)+",")

    file.close() 
    
    better_tag_val_data = Y_Val.copy()

    X_Train = np.array(X_Train)
    X_Val = np.array(X_Val)
    Y_train[Y_train != 0] = 1
    Y_Val[Y_Val != 0] = 1
     
    classifier = MLPClassifier(hidden_layer_sizes=(500), verbose=True, max_iter=30, warm_start=True)
    classifier.fit(X_Train, Y_train)
    
    simi = cdist(better_tag_val_data, classifier.predict_proba(X_Val), metric='cosine')
    pd.DataFrame(simi, columns=list(range(simi.shape[1]))).to_csv("Tunning/best.csv", index=False)
    
    model = Sequential()
    model.add(Dense(500, input_dim=X_Train.shape[1], init='normal', activation='relu'))
    model.add(Dense(Y_train.shape[1], init='normal', activation='softmax'))

    # Compile model     #logarithmic  loss     #method
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    # Fit the model
    model.fit(X_Train, Y_train, 
              nb_epoch=13, batch_size=100,
              verbose=1)

    predicted_tags = model.predict(X_Val)

    simi = cdist(better_tag_val_data, predicted_tags, metric='cosine')
    pd.DataFrame(simi, columns=list(range(simi.shape[1]))).to_csv("Tunning/NN.csv", index=False)
    
    return gt_files

In [23]:
from scipy.optimize import minimize

def optimize():
    def get_matrices(models):
        matrices = []
        for key in models:
            matrices.append(np.array(pd.read_csv("Tunning/{}.csv".format(key))))

        matrices = np.array(matrices)

        return matrices
    
    def get_accuracy_on_set(simi):
        
        gt_files = open("Tunning/gt.txt", "r").readlines()[0]
        
        gt_files = gt_files.split(",")[:-1]
        
        score = 0.0
        for i_check in range(len(simi)):
            pred_list = list(simi[i_check].argsort()[:20])

            k = [gt_files[i] for i in pred_list]
            if gt_files[i_check] in k:
                score += float(20 - k.index(gt_files[i_check]))/20

        return -score/2000
    
    matrices = get_matrices(["best", "NN"])
    
    def f(x):
        simi = x[0]*matrices[0] + x[1]*matrices[1]
        return get_accuracy_on_set(simi)

    start_pos = np.array([0.5, 0.5])

    res = minimize(f, start_pos, method='Nelder-Mead')
    coef = res.x
    print(coef)
    print(-get_accuracy_on_set(coef[0]*matrices[0] + coef[1]*matrices[1]))

In [24]:
# optimize()

## CNN

In [25]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img('data/images_train/0.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=2,
                          save_to_dir='Augmented', save_prefix='0', save_format='jpeg'):
    i += 1
    if i > 5:
        break  # otherwise the generator would loop indefinitely

In [26]:
X_train = []
for i in range(10000):
    img = load_img('data/images_train/{}.jpg'.format(i))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    X_train.append(x)

In [27]:
X_test = []
for i in range(2000):
    img = load_img('data/images_test/{}.jpg'.format(i))  # this is a PIL image
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
    X_test.append(x)

In [28]:
y = get_output(better=[True, "adj&noun&verb", 10])

In [29]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [30]:
X_train = X_train.reshape(X_train.shape[0], 3, 224, 224).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 3, 224, 224).astype('float32')

In [ ]:
def larger_model():
    # create model
    model = Sequential()
    model.add(Convolution2D(16, 3, 3, input_shape=(3, 224, 224), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

num_classes = y.shape[1]

# build the model
model = larger_model()

# Fit the model
model.fit(X_train, y, nb_epoch=5, batch_size=100, verbose=1)

Epoch 1/5
 3400/10000 [=========>....................] - ETA: 860s - loss: 55.7338 - acc: 0.0000e+00 

In [18]:
import h5py
filename = 'resnet101_weights_tf.h5'
f = h5py.File(filename, 'r')

# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])

Keys: KeysView(<HDF5 file "resnet101_weights_tf.h5" (mode r)>)


In [124]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        pool = x
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x, pool

def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model


model = resnet152(True)

In [115]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from torch.autograd import Variable
from torch import Tensor

images = []
for i in range(10000):
    img = load_img('data/images_train/{}.jpg'.format(i))
    im = img_to_array(img)  # this is a Numpy array with shape (3, 224, 224)

    # Remove train image mean
    im[:, :, 0] *= 1/255
    im[:, :, 1] *= 1/255
    im[:, :, 2] *= 1/255

#     im[:, :, 0] /= (0.225*255)
#     im[:, :, 1] /= (0.224*255)
#     im[:, :, 2] /= (0.229*255)

    # 0.229, 0.224, 0.225

    # Insert a new dimension for the batch_size
    im = np.expand_dims(im, axis=0)
    images.append(im)

In [137]:
output_fc_ls, output_pool_ls = [], []
k = 0
for im in images:
    output_fc, output_pool = model(Variable(Tensor(im)))
    output_fc_ls.append(output_fc.data.numpy().ravel())
    output_pool_ls.append(output_pool.data.numpy().ravel())
    k += 1
    if k % 50 == 0:
        print(k)

output_fc_ls = np.array(output_fc_ls)
output_pool_ls = np.array(output_pool_ls)



50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950
10000
[[-0.92519498 -0.34

AttributeError: 'NoneType' object has no attribute 'shape'

In [138]:
print(output_fc_ls.shape)
print(output_pool_ls.shape)

(10000, 1000)
(10000, 2048)


In [142]:
pd.DataFrame(output_fc_ls, columns=list(range(len(output_fc_ls[0])))).to_csv("ResNet152_fc_train.csv", 
                                                                          header=False, 
                                                                          index=False)
pd.DataFrame(output_pool_ls, columns=list(range(len(output_pool_ls[0])))).to_csv("ResNet152_pool_train.csv", 
                                                                          header=False, 
                                                                          index=False)

In [ ]:
images = []
for i in range(2000):
    img = load_img('data/images_test/{}.jpg'.format(i))
    im = img_to_array(img)  # this is a Numpy array with shape (3, 224, 224)

    # Remove train image mean
    im[:, :, 0] *= 1/255
    im[:, :, 1] *= 1/255
    im[:, :, 2] *= 1/255

    # Insert a new dimension for the batch_size
    im = np.expand_dims(im, axis=0)
    images.append(im)

In [ ]:
output_fc_ls, output_pool_ls = [], []
k = 0
for im in images:
    output_fc, output_pool = model(Variable(Tensor(im)))
    output_fc_ls.append(output_fc.data.numpy().ravel())
    output_pool_ls.append(output_pool.data.numpy().ravel())
    k += 1
    if k % 50 == 0:
        print(k)

output_fc_ls = np.array(output_fc_ls)
output_pool_ls = np.array(output_pool_ls)

print(output_fc_ls.shape)
print(output_pool_ls.shape)

pd.DataFrame(output_fc_ls, columns=list(range(len(output_fc_ls[0])))).to_csv("ResNet152_fc_test.csv", 
                                                                          header=False, 
                                                                          index=False)
pd.DataFrame(output_pool_ls, columns=list(range(len(output_pool_ls[0])))).to_csv("ResNet152_pool_test.csv", 
                                                                          header=False, 
                                                                          index=False)